# Corpusanalyse UiTdatabank

We hebben de volgende bibliotheken nodig:

In [1]:
from pandas import read_excel, read_csv, DataFrame, Series, concat
from datetime import datetime
from codecs import open
from re import compile, IGNORECASE
from json import dumps
from datetime import datetime

Dan lezen we lezen de gegevens in

In [2]:
df_podium = read_excel("ruwe data/podium.xlsx", sheetname='theaterdans1014')
df_bk = read_excel("ruwe data/beeldendekunsten.xlsx", sheetname='UitRapport')
df_muziek = read_excel("ruwe data/muziek.xlsx", sheetname='Int nat reg')

Voor de muziekgegevens moeten we nog controleren dat concerten eventuele herhalingen hebben, door in de speelmomenten kolom na te gaan wat de speelmomenten zijn. Dit is niet nodig bij de beeldende kunsten, aangezien daar de unit of analysis de tentoonstelling is, onafhankelijk van hoelang die tentoonstelling loopt. Bij podiumkunsten zijn de speelmomenten manueel gecheckt, ocharme simon.

In [3]:
datumregex = compile(r"\d\d/\d\d/\d\d")
df_muziek_expanded = df_muziek.copy()
for row in df_muziek_expanded.iterrows():
    speelmomenten = row[1]["Speelmomenten"]
    if str(speelmomenten) != "nan":    
        for speelmoment in datumregex.findall(speelmomenten):
            speelmoment_dt = datetime(int("20" + speelmoment.split("/")[2]), 
                                      int(speelmoment.split("/")[1]), 
                                      int(speelmoment.split("/")[0]))
            if speelmoment_dt != row[1]["Datum"]:
                df_muziek_expanded = df_muziek_expanded.append(
                    Series(
                        {"Discipline": row[1]["Discipline"],
                         "Tekst": row[1]["Tekst"],
                         "Datum": speelmoment_dt,
                         "Gemeente": row[1]["Gemeente"]
                        }, name=speelmoment_dt.isoformat() + " " + str(row[0])
                    )
                )

We perken de gegevens in tot 2014, en gooien ook duplicaten op basis van datum, gemeente en tekst eruit. Bovendien hebben we de kolom met speelmomenten ook niet meer nodig.

In [4]:
df_muziek_expanded_geenspeelmomenten = df_muziek_expanded.drop("Speelmomenten", axis=1)
df = concat([df_podium, df_bk, df_muziek_expanded_geenspeelmomenten])
df = df[df["Datum"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))]
df.drop_duplicates(subset=["Datum", "Discipline", "Gemeente", "Tekst"], inplace=True)

We kunnen kort inspecteren hoe deze data eruitzien.

In [5]:
df.head()

,Datum,Discipline,Gemeente,Tekst
747,2014-04-02,podium,Brussel,Shen Yun brengt 5.000 jaar Chinese beschaving ...
748,2014-04-03,podium,Brussel,Shen Yun brengt 5.000 jaar Chinese beschaving ...
749,2014-04-04,podium,Brussel,Shen Yun brengt 5.000 jaar Chinese beschaving ...
750,2014-04-05,podium,Brussel,Shen Yun brengt 5.000 jaar Chinese beschaving ...
752,2014-04-06,podium,Brussel,Shen Yun brengt 5.000 jaar Chinese beschaving ...


We zien dat elke lijn een event beschrijving bevat, de plaats waar een event plaatsvindt, en ook de datum. Merk op dat events die op meerdere dagen plaatsvinden een aparte lijn krijgen. We zullen hiermee rekening houden in de interpretatie van de resultaten.

Frequentielijst van woorden met een hoofdletter die na in of uit voorkomen, of na een lidwoord.

Voor onze analyse hebben we ook nood aan een lijst van namen van landen, coordinaten voor de plaatsnamen, en ook een manueel gemaakte mapping om de plaatsnamen in de UiTdatabank gegevens te normaliseren.

In [6]:
typering = read_csv("extra gegevens/mapping_udb-gemeente_fusie-gemeente.csv", delimiter=';')
coord = read_csv("extra gegevens/coordinaten.csv", delimiter=';')
landen = read_excel("extra gegevens/landen.xlsx", sheetname="uitgebreide lijst 2014")

Laten we even in detail deze tabellen bekijken. De landen:

In [7]:
landen.head()

,Continent (staatkundig),Land,Soort mention,Mention
0,Azië,Afghanistan,korte landnaam,Afghanistan
1,Azië,Afghanistan,inwoner,Afghaan
2,Azië,Afghanistan,inwoner,Afghanen
3,Azië,Afghanistan,adjectief,Afghaans
4,Azië,Afghanistan,adjectief,Afghaanse


Voor elk land weten we in welk (staatkundig) continent het ligt, en we hebben in de kolom 'Mention' verschillende manieren waarop dat land kan voorkomen in de tekst.

Bij typering zien we het volgende:

In [8]:
typering.tail()

,Gemeente Origineel,Postcode,Fusiegemeente,Stedelijkheid fusiegemeenten,Gemeente cluster,Province (English),Country (English),Provincie,Stedelijkheid
922,Impe (Lede),NaN,Lede,Niet-stedelijk,Lede,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
923,Herderen (Riemst),NaN,Riemst,Niet-stedelijk,Riemst,Limburg,Belgium,Limburg,Niet-stedelijk
924,Opgrimbie (Maasmechelen),NaN,Maasmechelen,Niet-stedelijk,Maasmechelen,Limburg,Belgium,Limburg,Niet-stedelijk
925,Mespelare (Dendermonde),NaN,Dendermonde,Niet-stedelijk,Dendermonde,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
926,Lombardsijde (Middelkerke),NaN,Middelkerke,Niet-stedelijk,Middelkerke,West Flanders,Belgium,West-Vlaanderen,Niet-stedelijk


De kolom Gemeente Origineel is de naam van de gemeente in de uitdatabank gegevens, en we kunnen de naam in de kolom Fusiegemeente en Province (English) gebruiken om een genormaliseerd zicht te krijgen.

Tot slot hebben we nog de coordinaten:

In [9]:
coord.head()

,Fusiegemeente,adress,latitude,longitude
0,Aalst,"Aalst, Belgium","50,9378101","4,0409517"
1,Aalter,"Aalter, Belgium","51,087349","3,448371"
2,Aarschot,"Aarschot, Belgium","50,9859959","4,8365218"
3,Aartselaar,"Aartselaar, Belgium","51,1340539","4,3844742"
4,Affligem,"Affligem, Belgium","50,9084","4,11281"


Hiermee kunnen we voor iedere Fusiegemeente (zie vorige tabel) de latitude en longitude ophalen.

Nu gaan we voor ieder event in de UiTdatabankgegevens na welk land er vermeld wordt in de beschrijving van dat event. We houden ook al onmiddellijk bij wat de genormaliseerde naam is van de gemeente en de coordinaten van het centrum. Bovendien tellen we binnen het land ook nog de verschillende disciplines.

In [10]:
data = {}
niet_vlaams = ["Jodoigne", "Tournai"]
count = 1
for row in list(df.iterrows()):
    if count % 100 == 0:
        print(count, "of", len(df.index))
    count += 1
    tekst = row[1]["Tekst"]
    gemeente = row[1]["Gemeente"]
    if str(gemeente) != "nan" and str(gemeente) not in niet_vlaams:
        for land in set(landen["Land"].values):
            for mention in landen[landen["Land"] == land]["Mention"]:
                regex = compile(r"\b" + mention + r"\b", IGNORECASE)
                if len(regex.findall(tekst)) > 0:
                    try:
                        typeringlijn = typering[typering["Gemeente Origineel"] == gemeente]
                        fusiegemeente = typeringlijn["Fusiegemeente"].values[0]
                        provincie = typeringlijn["Province (English)"].values[0]
                        lat = coord[coord["Fusiegemeente"] == fusiegemeente]["latitude"].values[0]
                        lon = coord[coord["Fusiegemeente"] == fusiegemeente]["longitude"].values[0]
                        continent = landen[landen["Mention"] == mention]["Continent (staatkundig)"].values[0]
                        mention_soort = landen[landen["Mention"] == mention]["Soort mention"].values[0]
                        discipline = row[1]["Discipline"]
                        if fusiegemeente not in data:
                            data[fusiegemeente] = {"latitude": float(lat.replace(',', '.')),
                                                   "longitude": float(lon.replace(',', '.')),
                                                   "provincie": provincie,
                                                   "vermeldingen": {}
                                                   }
                        if continent not in data[fusiegemeente]["vermeldingen"]:
                            data[fusiegemeente]["vermeldingen"][continent] = {}
                        if land not in data[fusiegemeente]["vermeldingen"][continent]:
                            data[fusiegemeente]["vermeldingen"][continent][land] = {}
                        if discipline not in data[fusiegemeente]["vermeldingen"][continent][land]:
                            data[fusiegemeente]["vermeldingen"][continent][land] = {discipline: 0}
                        data[fusiegemeente]["vermeldingen"][continent][land][discipline] += 1
                    except:
                        print(gemeente)

100 of 32968
200 of 32968
300 of 32968
400 of 32968
500 of 32968
600 of 32968
700 of 32968
800 of 32968
900 of 32968
1000 of 32968
1100 of 32968
1200 of 32968
1300 of 32968
1400 of 32968
1500 of 32968
1600 of 32968
1700 of 32968
1800 of 32968
1900 of 32968
2000 of 32968
2100 of 32968
2200 of 32968
2300 of 32968
2400 of 32968
2500 of 32968
2600 of 32968
2700 of 32968
2800 of 32968
2900 of 32968
3000 of 32968
3100 of 32968
3200 of 32968
3300 of 32968
3400 of 32968
3500 of 32968
3600 of 32968
3700 of 32968
3800 of 32968
3900 of 32968
4000 of 32968
4100 of 32968
4200 of 32968
4300 of 32968
4400 of 32968
4500 of 32968
4600 of 32968
4700 of 32968
4800 of 32968
4900 of 32968
5000 of 32968
5100 of 32968
5200 of 32968
5300 of 32968
5400 of 32968
5500 of 32968
5600 of 32968
5700 of 32968
5800 of 32968
5900 of 32968
6000 of 32968
6100 of 32968
6200 of 32968
6300 of 32968
6400 of 32968
6500 of 32968
6600 of 32968
6700 of 32968
6800 of 32968
6900 of 32968
7000 of 32968
7100 of 32968
7200 of 32968
7

We kunnen al eens gauw de structuur van deze gegevens nakijken voor bijvoorbeeld de gemeente Brussel:

In [11]:
print(dumps(data["Brussel"], indent=2))

{
  "latitude": 50.8503396,
  "provincie": "Brussels",
  "vermeldingen": {
    "Afrika": {
      "Somali\u00eb": {
        "podium": 9
      },
      "Rwanda": {
        "Tentoonstelling": 1
      },
      "Kameroen": {
        "Tentoonstelling": 1
      },
      "Algerije": {
        "Tentoonstelling": 3
      },
      "Egypte": {
        "Concert": 3
      },
      "Zimbabwe": {
        "Concert": 2
      },
      "Ethiopi\u00eb": {
        "Concert": 2
      },
      "Nigeria": {
        "Concert": 7
      },
      "Soedan": {
        "Tentoonstelling": 1
      },
      "Kenia": {
        "Tentoonstelling": 1
      },
      "Mauritius": {
        "podium": 1
      },
      "Ghana": {
        "Tentoonstelling": 1
      },
      "Togo": {
        "Concert": 1
      },
      "Ivoorkust": {
        "Concert": 1
      },
      "Zuid-Soedan": {
        "Tentoonstelling": 1
      },
      "Seychellen": {
        "Tentoonstelling": 1
      },
      "Marokko": {
        "Concert": 4
      },

We kunnen nu deze gegevens samenballen in een tabel.

In [15]:
lines = []
for fusiegemeente in data:
    print(fusiegemeente)
    lat = data[fusiegemeente]["latitude"]
    lon = data[fusiegemeente]["longitude"]
    prov = data[fusiegemeente]["provincie"]
    for continent in data[fusiegemeente]["vermeldingen"]:
        for land in data[fusiegemeente]["vermeldingen"][continent]:
            for discipline in ["Concert", "Tentoonstelling", "podium"]:
                if discipline in data[fusiegemeente]["vermeldingen"][continent][land]:
                    lines.append([fusiegemeente, lat, lon, prov, continent, land, discipline, 
                                  data[fusiegemeente]["vermeldingen"][continent][land][discipline]])
                else:
                    lines.append([fusiegemeente, lat, lon, prov, continent, land, discipline, 0])
res = DataFrame(lines, columns=['Gemeente', 'Latitude', 'Longitude', 'Provincie', 'Continent', 'Land', 'Frequentie'])

Asse
Steenokkerzeel
Ichtegem
Hulshout
Overijse
Hoeilaart
Veurne
Lede
Hamme
Wielsbeke
Borgloon
Wetteren
Wervik
Wezembeek-Oppem
Bornem
Hasselt
Staden
Merelbeke
Dessel
Vorst
Koekelare
Tielt
Evergem
Wortegem-Petegem
Kampenhout
Lier
Kruishoutem
Baarle-Hertog
Beveren
Tervuren
Brakel
Herenthout
Herentals
Tessenderlo
Anderlecht
Maldegem
Westerlo
Alken
Zwijndrecht
Bilzen
Turnhout
Brugge
Linkebeek
Zulte
Mol
Londerzeel
Zottegem
Deinze
Ravels
Lichtervelde
Haaltert
Beerse
Zoersel
Tienen
Kruibeke
Schelle
Lierde
Oudenburg
Evere
Nevele
Destelbergen
Sint-Genesius-Rode
Schilde
Rijkevorsel
Hamont-Achel
Kuurne
Keerbergen
Retie
Essen
Neerpelt
Beersel
Haacht
Bree
Stabroek
Jabbeke
Riemst
Bredene
Jette
Wommelgem
Waasmunster
Mortsel
Aalst
Knesselare
Nieuwpoort
Middelkerke
Meulebeke
Sint-Martens-Latem
Halle
Kortenaken
De Pinte
Vilvoorde
Grobbendonk
Anzegem
Vorselaar
Sint-Joost-Ten-Node
Lokeren
Kortemark
Berlare
Opwijk
Zedelgem
Geel
Hemiksem
Meerhout
Rotselaar
De Panne
Diest
Ardooie
Ranst
Ternat
Lebbeke
Heusden-

AssertionError: 7 columns passed, passed data had 8 columns

De tabel bevat dus de volgende informatie:

In [ ]:
res.head()

Als voorbeeld kan de eerste lijn gelezen worden als "in de gemeente Denderleeuw (coordinaten 50.88, 4.07) wordt India 1 keer vermeld".

We schrijven tot slot deze tabel uit naar een CSV bestand voor verdere analyse.

In [ ]:
res.to_csv("landsvermeldingen.csv")